In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D,BatchNormalization, Activation
import glob

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
import glob

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

target_size_dim = 300

batch_size = 64

In [ ]:
test_images = glob.glob('../input/ranzcr-clip-catheter-line-classification/test/*.jpg')
df_test = pd.DataFrame(np.array(test_images), columns=['Path'])
df_test.head()

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((df_test.Path.values))


def process_test(image_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [target_size_dim,target_size_dim])
    return img
    
test_ds = test_ds.map(process_test, num_parallel_calls=AUTOTUNE).batch(batch_size*2)

In [ ]:
for image in test_ds.take(1):
    plt.imshow(image[0].numpy().astype('uint8'))
    plt.show()
    print("Image shape: ", image.numpy().shape)
    

In [ ]:

weight_path = '../input/efficientnetb3-tf2-keras-baseline/best_model.hdf5'
my_model = load_model(weight_path)


In [ ]:
pred_y = my_model.predict(test_ds, workers=4, verbose=1)

In [ ]:
labels = ['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

In [ ]:
df_ss = pd.DataFrame(pred_y, columns = labels)
df_test['image_id'] = df_test.Path.str.split('/').str[-1].str[:-4]
df_ss['StudyInstanceUID'] = df_test['image_id']
df_ss.head()

In [ ]:
cols_reordered = ['StudyInstanceUID', 'ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal',
       'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
       'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
       'Swan Ganz Catheter Present']

df_order = df_ss[cols_reordered]

In [ ]:
df_order.head()

In [ ]:
df_order.to_csv('submission.csv', index=False)